In [ ]:
# !pip install telebot
!pip install pyTelegramBotAPI
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install nltk

import telebot
import threading
import time
import cv2
import numpy as np
from PIL import Image as im
import io
from telebot import types
import requests
from pytesseract import pytesseract
from nltk.corpus import words
import nltk

nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

bot = telebot.TeleBot('*bot key here*')
#usage of tesseract for text recognition
pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

class Constants(object):
  ping = "ping"
  file_name = "/content/image.png"
  endpoint = 'http://*MODEL ENDPOINT ADDRESS HERE*:8502/predict/'

@bot.message_handler(commands=['start'])
def welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard = True)
    item1 = types.KeyboardButton(Constants.ping)

    markup.add(item1)

    bot.send_message(
        message.chat.id,
         """
         I am ImgToTex.\nI am created to transform images with formulas to TEX.
          Just send photos to the chat and I will transform them to TEX.")
        """
        .format(message.from_user, bot.get_me()),
        parse_mode = 'html',
        reply_markup = markup
      )


@bot.message_handler(commands=['restart'])
def rewelcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard = True)
    item1 = types.KeyboardButton(Constants.ping)

    markup.add(item1)
    bot.send_message(
        message.chat.id,
        """
        I am ImgToTex.\nI am created to transform images with formulas to TEX.
        Just send photos to the chat and I will transform them to TEX.
        """
        .format(message.from_user, bot.get_me()),
        parse_mode = 'html',
        reply_markup = markup
      )

@bot.message_handler(content_types=['text', 'photo', 'document'])
def handleMessage(message):
    if message.chat.type == 'private':
        if message.text == Constants.ping:
            ping(message)
        else:
            download_image(message)

def ping(message):
    bot.send_message(message.chat.id, 'pong')

def download_image(message):
    if message.photo is not None:
      photo = download_photo(message)
      prediction = predict(message, photo)
      # prediction = pretty(prediction)
      bot.send_message(message.chat.id, prediction)
    elif message.document is not None:
      file = download_file(message)
      prediction = predict(message, file)
      # prediction = pretty(prediction)
      bot.send_message(message.chat.id, prediction)
    else :
      bot.send_message(message.chat.id, "Incorrect file format.")

def download_photo(message):
    bot.send_message(message.chat.id, "Loading image..")
    raw = message.photo[0].file_id
    file_info = bot.get_file(raw)
    image = bot.download_file(file_info.file_path)
    bot.send_message(message.chat.id, "Image loaded.")
    return image

def download_file(message):
    bot.send_message(message.chat.id, "Loading file..")
    file_info = bot.get_file(message.document.file_id)
    file = bot.download_file(file_info.file_path)
    bot.send_message(message.chat.id, "File loaded.")
    return file

def predict(message, file):
    bot.send_message(message.chat.id, "Starting prediction..")

    # response = requests.post(Constants.endpoint, files=dict(file=file))
    # txt = pretty(response)
    # bot.send_message(message.chat.id, txt)

    with open(Constants.file_name, 'wb') as new_file:
        new_file.write(file)
    
    img = cv2.imread(Constants.file_name)
    
    lines = getLines(img, 0)
    bot.send_message(message.chat.id, "Got "+str(len(lines))+" lines")
    msg = ""
    for i in range(len(lines)):
      cv2.imwrite("line_"+str(i)+".png", lines[i])

      img = im.open("line_"+str(i)+".png")
      text = pytesseract.image_to_string(img)

      if checkForWords(text):  
        msg += text + "\n"
      else:
        _, bts = cv2.imencode('.webp', lines[i])
        bts = bts.tobytes()
        response = requests.post(Constants.endpoint, files=dict(file=bts))
        txt = pretty(response)
        msg += txt + "\n\n"
        # if "begin{array}" in txt:
        #   img = cv2.imread("line_"+str(i)+".png")
        #   elements = getLines(img, 2)
        #   for j in range(len(elements)):
        #     cv2.imwrite("element_line_"+str(j)+".png", elements[j])
        #     # img = im.open("line_"+str(i)+".png")
        # msg += txt + "\n\n"
        
      
    bot.send_message(message.chat.id, msg)
    return "Prediction completed."

def checkForWords(text):
  splitted = nltk.word_tokenize(text)
  # print(splitted)
  for word in splitted:
    word = word.lower()
    # print(word, word in words.words(), word not in stopwords.words())
    if word in words.words() and word not in stopwords.words():
      return True
  return False

def getLines(img, threshold):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
  hist = cv2.reduce(threshed,1, cv2.REDUCE_AVG).reshape(-1)

  th = threshold
  H,W = img.shape[:2]
  uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
  lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]
  colored = cv2.cvtColor(threshed, cv2.COLOR_GRAY2BGR)
  
  if threshold != 0:
    for i in range(1, 50, 5):
      th = i
      H,W = img.shape[:2]
      uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
      lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]
      print(i, uppers, lowers)
      colored = cv2.cvtColor(threshed, cv2.COLOR_GRAY2BGR)
      drawLines(colored, uppers, lowers, i)

  lines = []
  min_font_size = 24
  if len(lowers) > len(uppers):
    if lowers[0] < min_font_size:
      lowers.pop(0)
    else:
      uppers.appendleft(0)
  elif len(lowers) < len(uppers):
    if uppers[-1] > H - min_font_size:
      uppers.pop()
    else:
      lowers.append(H-1)
  
  bounds = []
  line_err = min_font_size / 2
  for i in range(0,len(uppers)):
    if len(bounds) == 0 or len(bounds[-1]) == 2:
      bounds.append( [uppers[i]] )

    if i + 1 != len(uppers) and uppers[i+1] - lowers[i] < line_err:
      continue
    bounds[-1].append(lowers[i])

  for bound in bounds:
    upper_bound = max(0, bound[0] - 3)
    lower_bound = min(bound[1] + 3, H-1)
    lines.append(colored[upper_bound:lower_bound])

  return lines

def drawLines(img, uppers, lowers, i):
  H,W = img.shape[:2]
  for y in uppers:
      cv2.line(img, (0,y), (W, y), (255,0,0), 1)

  for y in lowers:
      cv2.line(img, (0,y), (W, y), (0,255,0), 1)
  
  cv2.imwrite("result_drawn_lines"+str(i)+".png", img)

def pretty(response):
  text = str(response.text)
  text = remove_response_body(text)
  text = remove_backslashes(text)
  return text

def remove_response_body(text):
  return text[1:-1]

def remove_backslashes(text):
  text = str(text).replace('\\\\', '\\')
  return text

bot.polling()